# Title: The dictatorship of consortia on the media they own.

# Le plan: 

When one wants to get information 
## 1. Blabla intro du sujet TODO

**idée: "you reader, try and find your favourite US-journal(s)/media(s) in our cluster"**

## 2. What is our goal?

## 3. How do we achieve it (in short)?
~Use Quotebank + Unsupervised learning

## 4. Data story
**TODO: voir à quel moment mentionner le bokeh plot**

## 5. Caveats (ou inverser 5. et 6. idk)

## 6. Conclusions: 
### 6.1 What did we find?
### 6.2 What's next/would be interesting to do

### somewhere:
Indicate that we interchangeably use the terms speaker/citee/quotee, journal/media/newspaper, and group/consortium

# 1. Intro du sujet:
**TODO**

One's view of the world is based on the information we have on it, and how it is conveyed. A big part of that information we obtain from the media, and in particular "traditional media" namely newspapers such as The Guardian or Fox News. As a result, these media have a huge impact on the public’s opinion.

One way to ensure you are well-informed (i.e. you are getting information on various topics) and are
not biased towards a particular stance[^1] is to gather knowledge from several sources,
e.g. several media.

[^1]: https://www.allsides.com/media-bias/media-bias-ratings

But a lot of these media are owned by the same groups. This begs the question whether
the information we are getting really is diverse, or if the news and opinions shared
are roughly the same for media belonging to the same entity.[^2]

[^2]: https://www.youtube.com/watch?v=_fHfgU8oMSo

To answer that question, we will try to cluster journals according to different metrics and features in order to evaluate the opinions they defend. Other scientists have already obtained
similar results[^3] [^4] based on twitter which makes us also expect to get interesting results.

[^3]: https://www.mediaobservatory.com/
[^4]: https://newsteller.io/source

# 2. Our goal(s):
We aim to show that we can recognize which group some media belong to, using very little data.  
As a result, we will have shown a lack of diversity of these media, thus showing that if you want to be well-informed, simply reading different newspapers is not enough. One needs to pay attention to more than that.

# 3. How we achieve that[^1]:
[^1]: For a detailed step-by-step description of the process, plase refer to [our notebook](https://github.com/epfl-ada/ada-2021-project-mahj)

### 3.1: Overview of the process

We need some *features* which allow us to distinguish between diferent media. To do so we use the [Quotebank](https://dlab.epfl.ch/people/west/pub/Vaucher-Spitz-Catasta-West_WSDM-21.pdf) database, which is a dataset of  speaker-attributed quotations that were extracted from diverse English news articles[^2].

[^2]: https://zenodo.org/record/4277311

We then augment this dataset using [Wikidata](https://www.wikidata.org/wiki/Wikidata:Main_Page) to get the features we deemed necessary for our analysis.

 

### 3.2: Explanation of some of our choices and challenges
We have chosen to only focus on one year (namely 2020), as we expect the topics of interest, as well as the prominent people to vary from one year to another.

We expect basically all the media to talk about coronavirus, so we need to make sure it does not act as a "*great equalizer*" that overshadows other differences between media.

As the data is so big, it cannot be held in the RAM all at once. Therefore we had to implement some pipelines treating the data chunk by chunk.

# 4. But more precisely, what did you do?:
**TODO: sur les graphes préciser que l'axe indiqué en premier correspond aux abscisses.**

### 4.1: data-augmentation[^1]
[^1]: refer to file Wikidata_scraping.ipynb for more details  

The data we extract from the Quotebank-dataset look like this:

In [10]:
# Ici quelques extraits de la data de quotebank avec juste les attributs qu'on a gardés

# Ou ptetre ?:git 
 # 1: original
 # 2: ce qu'on a gardé

As you can see, the owners of the media are not indicated here, so we need to extend our dataset with these features.  
If we can't automatically find a journal's parent group, we decide to throw it away.
We also do so if the journals are lonely children. Indeed, we seek first and foremost to cluster them according to their parent groups, so isolated media are irrelevant.

In [1]:
# Exemple de à quoi ressemble les infos obtenues / la nouvelle data

We also add the country of origin of the media, because we expect to see differences in media based on their country of origin.

### 4.2: Data processing[^1]?
Since for each we have both quotes, and quotees (i.e. the people quoted), it seems natural to try and cluster them according to these 2 features. Let us then try 2 different methods:
 * First, cluster according to which speakers are cited in it.
 * Second, cluster according to the text content of the extracted quotes.
 
[^1]: see file "Process_data.ipynb" for more details  


In [9]:
# For 1, do a small graph like:
# article i:
  # speaker 1
  # speaker 2
  # ...
  # speaker n

# For 2, do sth like:
# journal j:
  # quote 1
  # quote 2
  # ...
  # quote m

# Maybe 1 on the left, and 1 on the right.

For both these features, we will first use the [Term Frequency-Inverse Document Frequency](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) method (TF-IDF for short) before proceeding to further analysis.  
In a nutshell, this method allows one to measure the importance of a term[^1] to a document[^2] in a collection[^3] based on its number of apparitions.

[^1]: Here a citee, or a word
[^2]: Here a media
[^3]: Here, all the media we have kept

More clearly, this method gives to each word a weight:
 * *proportional* to its number of occurences in d (term frequency)
 * and *inversely proportional* to its presence in the corpus (inverse document frequency)[^1]
[^1]: to quantify the 'presence' of a word in the corpus, we simply count the number of documents in which it occurs. We don't take its occurences per document into account here

The reasoning is that we use the **IDF** to **find** words **unique** to each document, or **remove recurrent** words, which we find in basically every document, and therefore don't bring any information (e.g. stopwords such as 'the' in a text, or terms related to covid-19[^2]).  
We use the **TF** to find words that **appear often**, as they may be **characteristic** of a particular document. For example the word 'brain' may often appear in a neuroscience paper, while not so much in a geography one.

[^2]: so this solves, or at least mitigates the issue mentione in **3.2**

In [14]:
# Illustrations: les formes des 2 matrices obtenues, peu importe les coefs

# Though on peut mettre une petit flèche vers
# la matrice journal-speaker[i, j] par ex.

### 4.3: Data analysis (the heart of our problem)

Now that we have processed our data, we need to visualize it.  
The problem is that the data (our TF-IDF matrices) is inherently high-dimensional, so we as humans can't directly visualize it. That is why we transform our matrices using [Singular Value Decomposition](https://en.wikipedia.org/wiki/Singular_value_decomposition) (SVD).

This transformation should allow us to extract "[topics](https://en.wikipedia.org/wiki/Topic_model)", which correspond to the directions of highest variation, i.e. **the ones separating the media the best**, and their importance. We can then view the differences between different media along one such axis.

In [13]:
# Some illustration of SVD, or maybe of PCA to get the idea?

#### 4.3.1: The speakers
Let us apply this technique on the speakers matrix first.

In [17]:
# Illustration des projections qu'on a au départ,
# avec la séparation géographique.

Using the top speakers per axe, we identify the *topics* of the axes:  
**TODO: we have min and max, but isn't the axis's name base on max ?**
**e.g. first axis is "speaker frequency"

| Axis | Max                 | Min                     |
|------|---------------------|-------------------------|
| 0    | Frequent speakers   | Non Frequent speakers   |
| 1    | US speakers         | UK speakers             |
| 2    | Australian speakers | US speakers             |
| 3    | Canadian speakers   | US politician speakers  |
| 4    | Basketball speakers | US politicians speakers |

As you can see, the speakers are very different in newspapers from different countries which results in clustering mostly based on the country of origin of the newspapers.  
It makes sense: UK-media will cite more UK-speakers, while USA-media will cite more USA-speakers. 

So our clustering works (for example it is easy to recognize journals from the media group *News Corp Australia*), but it is not enough yet. 
Indeed, even if one is aware they should get their information from different media, are they really that likely to get it from different countries' media? Of course not, and that might be especially true for American people (**TODO: is there some study that says american are self centered, i.e. they get their info from american newspapers?**). 



Then let us see if we can differentiate the media *inside one country!*. We will focus on the USA since it is the country with the most data available

In [16]:
# show the list of plots "speaker with only USA"

Again we can try and extract the *topics* from the list of top speakers per axe:

| Axis | Max                             | Min                                              |
|------|---------------------------------|--------------------------------------------------|
| 0    | Frequent speakers               | Non Frequent speakers                          |
| 1    | Sport coaches of Los Angeles    | West coast personalities                       |
| 2    | Sport coaches of San Francisco  | Sport coaches/DJ of Los Angeles                |
| 3    | Nashville country singer        | Democrate political figure of Illinois/Minnesota |

We also see (in bright green) that journals from *Townsquare Media* explain most of the variance of several axes, namely the 3rd and the 4th.  
Hence although we can visually recognize media from this group, it might be interesting to redo our analysis while removing media from this group.

In [20]:
# les nouveaux plots

Clearly we can spot some clusters when projecting on the first few axes. The purple and yellow ones are especially well separated. 
**TODO: noms des groupes**

**TODO: without townsquare**

**TODO: la semi-conclusion on the efficiency of this technique, moment important du notebook:**

So we are able to recognize which group some journals belong by only using the speakers quoted as a feature.

Nevertheless, the most obvious separations seem to stem from geographical differences. We wonder whether through the use of words used in quotes we can cluster the media more efficiently. **todo: ~speakers are local, while words used are more subject-centered**

#### 4.3.2:  The quotes

Now let us apply this technique on the words' matrix (constructed with the words from the quotes).  
For starters we will consider all the quotes, and not only the ones in US newspapers. We expect there is still some geographical distinction --due for example to cultural differences, or diverging points of interest (e.g. American football is popular in the USA, but not much in other English speaking countries)--  

But we don't expect them to be as obvious as with the speakers' matrix, since we expect more recoupement between journals of different countries.

In [3]:
# Illustration des projections qu'on obtient

# ptetre un zoom sur le 0-4? ou un où on voit le mieux une séparation

Again, using the top words per axe we can identify the topics corresponding to the axes:

| Axis | Max            | Min                   |
|------|----------------|-----------------------|
| 0    | Frequent words | Non frequent words    |
| 1    | Sports[^1]     | US health             |
| 2    | UK **TODO: verify**             | Not clear. Some terms related to music though|
| 3    | Music          | Sports                |
| 4    | Canada         | US politics **TODO: verify**           |

Our hypothesis seems verified, as there are still some geographical distinctions, but not as obvious from the graphs[^2], or as numerous as previously.  

**TODO: a word on why "Sports" is both a min and a max on different axes**  

[^1]:we hypothesize American football in particular, but are not 100% certain.  
[^2]: groups of points of one particular color, i.e. corresponding to one consortium in particular are not as obvious when projected on 2 axes, although some still appear (e.g. strange green **TODO** color on projection on axes 0-4).

We now focus only on the USA again:

In [6]:
# Illustration des graphes avec les USA.

# zoom sur 0-4 par ex (pour les vert fluo, qui doivent correspondre à Townsquare Media)

As always, we try and identify the topics corresponding to the first axes:

| Axis | Max                   | Min                |
|------|-----------------------|--------------------|
| 0    | Frequent words        | Non frequent words |
| 1    | Sports                 | Public health      |
| 2    | Music                 | Sports              |
| 3    | Local / State related | presidential/political **TODO: verify**       |
| 4    | ~Montana state        | not clear **TODO: verify**          |

As you can see, they are similar to the previous ones --with unsurprisingly no "country separation axis" anymore.

Visually, the bright green points really stand out.  
Can you guess to which group they correspond?

<details>
  <summary>Click to reveal</summary>
  
  Once again, it is Townsquare media which stands out. That's interesting: it is not only divergent with respect to its choice of speaker, but also w.r.t. the subjects it treats.  
  We hypothesize it is so particular because it is composed of many *local radio stations*.
  Once more we will remove them to refine our analysis.
</details>

Let us remove this consortium from the analysis once again, and see what results we get:

In [1]:
# illustration of analysis on words, USA, without Townsquare

# zoom on 2-4 or 0-4 or 1-4 (yellow) (2-4 seems best), and maybe 0-2(pale green)

The axes we identify are as follow:

| Axis | Max                   | Min                |
|------|-----------------------|--------------------|
| 0    | Frequent words        | Non frequent words |
| 1    | Sport                 | Public health      |
| 2    | Music                 | Sport              |
| 3    | Local / State level | Country wide       |
| 4    | Election              | Covid              |

We can note than axes 0, 1 and 2 are identical to the previous ones.
Axis 3 is quite similar also, but if you delve into the details of the words[^1] you can see they are less related to specific states (e.g. words "Wyoming" or "Montana"), but rather to states in general.

The projections are now clearer (especially projection 1-4), and that for 2 reasons mainly:
 1. The withdrawal of articles from townsquare media, which had a high variance among several axes. It removed their influence, and hence the SVD projection could take into account the other media more.
 2. The removal of some points makes it easier to see the rest of them.

[^1]: see notebook "Analysis.ipynb"

**TODO: un mot sur les différences d'efficacité entre speaker-clustering et word-clustering? accompagné pk pas de 2 graphiques pour les opposer.**

From all these projections we can **todo: synonym of exhibit/identify** basically 3 types of media:

1. Media belonging to some groups e.g. **TODO** *w* are clustered together on axes *x* and *y*, and are therefore easily identifiable. 

2. There are also media belonging to a group which we can't as easily uniquely identify, for example because they are part of a bicolor cluster (**TODO: e.g. bla and blou on graphs *x'* and *y'***. Still, we can exhibit a trend on these few groups.

3. And finally, there are a number of groups which we have not managed to visually separate through these 2D-projections --see groups whose media are always in the middle of some big blob, e.g. *a*, or seem split all over, e.g. *b*)

The fact that a process as simple (**TODO: une astérisque/footnote où on explique en quoi il est simple dans la suite, ou on écrit simplement l'explication plus bas?**) as ours is able to exhibit media of groups 1 and 2 mean that these media have obvious bias (**TODO: make sure they are not together bc they are niche?**) which is correlated to the group to which they belong. This bias may be speaker-wise, where some people are **TODO: "mis en avant"** more than they should, or word-wise, where **TODO: x**.  

In any case, if an unaware viewer gets their information from media in this group, whether one or several, they are bound to be biased too.

**TODO: the astérisque**  
We deem our process as simple because:
 * It uses very little information, namely only
     * the names of the people quoted in articles of the media
     * or the words used in all quotes in articles of a media, seen as a **non-ordered** set.

   which furthermore is esaily accessible; you can get it either by yourself, or using some preexisting dataset such as the one we used: *Quotebank*. 
 * The algorithms on which it relies are conceptually relatively simple, which we described earlier:
     * the TF-IDF transformation, which we explained earlier
     * the Singular Value Decomposition.

# 6. Caveats and outlook

### 6.1 Caveats 
**TODO: mieux: là ça tombe un peu comme un cheveu sur la soupe**  
While our analysis made some biases clear, it is deeply linked to the dataset we used --here, *Quotebank*. That is why it is important to know how how the latter was generated. We advise one uses **TODO: "some peer-reviewed database"**. As the saying goes: *garbage in, garbage out*.

### 6.2 Outlook (and what's next)

 * better projections/visualization ->**TODO: expliquer comment?**.
 * more semantic analysis for quotes (better than simple bag/set of words)